In [2]:
from roboflow import Roboflow
rf = Roboflow(api_key="Ob9mLrO3fKqDwCHJu04j")
project = rf.workspace("surfline").project("surfer-spotting")
version = project.version(2)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Surfer-Spotting-2 in yolov11:: 100%|██████████| 71100/71100 [00:05<00:00, 13332.94it/s]


In [7]:
import os
from ultralytics import YOLO
import yaml 
import torch
EXISTING_DATA_YAML_PATH = "/home/user/Documents/ECE140/SurfWatch/AI/train/Surfer-Spotting-2/data.yaml"
PROJECT_NAME = "YOLOv11_Surfer_Spotting"

RUN_NAME = "yolov11m_finetune_gpu" 


MODEL_NAME = "yolo11m.pt" 
EPOCHS = 1 
BATCH_SIZE = 16 
IMG_SIZE = 640  
if torch.cuda.is_available():
    DEVICE = "cuda"
else:
    DEVICE = "cpu"
print(f"Using device: {DEVICE}")

if not os.path.exists(EXISTING_DATA_YAML_PATH):
    print(f"Error: Cannot find data.yaml at: {EXISTING_DATA_YAML_PATH}")
    print("Please ensure the path is correct.")
    exit()
else:
    print(f"Using existing data configuration: {EXISTING_DATA_YAML_PATH}")
print(f"Loading pre-trained model: {MODEL_NAME}")
model = YOLO(MODEL_NAME)


Using device: cpu
Using existing data configuration: /home/user/Documents/ECE140/SurfWatch/AI/train/Surfer-Spotting-2/data.yaml
Loading pre-trained model: yolo11m.pt


In [ ]:
print(f"Starting training on device: {DEVICE}...")
try:
    results = model.train(
        data=EXISTING_DATA_YAML_PATH, 
        epochs=EPOCHS,
        imgsz=IMG_SIZE,
        batch=BATCH_SIZE,
        project=PROJECT_NAME,
        name=RUN_NAME,
        device=DEVICE, 
    )
    print("Training finished.")
    print(f"Results saved to: {results.save_dir}")

except Exception as e:
    print(f"An error occurred during training: {e}")
    if "CUDA" in str(e):
        print(
            "A CUDA-related error occurred. Ensure CUDA is installed correctly, "
            "your NVIDIA drivers are up to date, and PyTorch with CUDA support is installed."
        )
        print(
            "You can check PyTorch CUDA availability with: python -c 'import torch; print(torch.cuda.is_available())'"
        )

#Eval
print(
    f"\nAttempting evaluation on the test set (if defined in YAML) using device: {DEVICE}..."
)
try:
   
    best_model_path = os.path.join(
        PROJECT_NAME, RUN_NAME, "weights", "best.pt"
    )
    if os.path.exists(best_model_path):
        print(f"Loading best weights from: {best_model_path}")
        model = YOLO(best_model_path)
        metrics = model.val(
            data=EXISTING_DATA_YAML_PATH, split="test", device=DEVICE
        )

        print("Test Set Evaluation Metrics:")

        print(f"  mAP50-95: {metrics.box.map:.4f}")
        print(f"  mAP50: {metrics.box.map50:.4f}")
        print(f"  mAP75: {metrics.box.map75:.4f}")
        
    else:
        print(
            f"Could not find best model weights at {best_model_path}. Skipping test set evaluation."
        )
except Exception as e:
    print(f"An error occurred during test set evaluation: {e}")
    print(
        "Ensure the 'test' key and path are correctly specified in your data.yaml if you want to evaluate."
    )


print("\nScript finished.")


Starting training on device: cpu...
Ultralytics 8.3.135 🚀 Python-3.12.10 torch-2.7.0+cu126 CPU (AMD Ryzen 7 5800HS with Radeon Graphics)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/user/Documents/ECE140/SurfWatch/AI/train/Surfer-Spotting-2/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov11m_finetune_gpu, nbs=64, nms=False, opset=None, o

100%|██████████| 755k/755k [00:00<00:00, 11.9MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  1    111872  ultralytics.nn.modules.block.C3k2            [128, 256, 1, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  1    444928  ultralytics.nn.modules.block.C3k2            [256, 512, 1, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  1   1380352  ultralytics.nn.modules.block.C3k2            [512, 512, 1, True]           
  7                  -1  1   2360320  ultralytics

  8                  -1  1   1380352  ultralytics.nn.modules.block.C3k2            [512, 512, 1, True]           
  9                  -1  1    656896  ultralytics.nn.modules.block.SPPF            [512, 512, 5]                 
 10                  -1  1    990976  ultralytics.nn.modules.block.C2PSA           [512, 512, 1]                 
 11                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 12             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 13                  -1  1   1642496  ultralytics.nn.modules.block.C3k2            [1024, 512, 1, True]          
 14                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 15             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 16                  -1  1    542720  ultralytics.nn.modules.block.C3k2            [1024

train: Scanning /home/user/Documents/ECE140/SurfWatch/AI/train/Surfer-Spotting-2/train/labels... 31104 images, 19032 backgrounds, 0 corrupt: 100%|██████████| 31104/31104 [00:27<00:00, 1128.79it/s]

...

val: /home/user/Documents/ECE140/SurfWatch/AI/train/Surfer-Spotting-2/valid/images/SanClementeStateBeach_2021_02_19_1224pm_frame_16_right_jpg.rf.66096cfb0218aa39c88aa4d70cb29641.jpg: 1 duplicate labels removed


val: New cache created: /home/user/Documents/ECE140/SurfWatch/AI/train/Surfer-Spotting-2/valid/labels.cache


/home/user/Documents/ECE140/SurfWatch/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Plotting labels to YOLOv11_Surfer_Spotting/yolov11m_finetune_gpu/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 106 weight(decay=0.0), 113 weight(decay=0.0005), 112 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to YOLOv11_Surfer_Spotting/yolov11m_finetune_gpu
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1         0G      2.845      7.178      1.241         82        640:   0%|          | 3/1944 [01:29<16:04:14, 29.81s/it]